## Sections
 * Import Datasets
 * EDA -- break into dimensions
 * Data Cleaning & Feature Engineering
 * Models

## Import Datasets

In [433]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [434]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import r2_score

In [435]:
assessments = pd.read_csv("assessments.csv")
courses = pd.read_csv("courses.csv")
studentAssessment = pd.read_csv("studentAssessment.csv")
studentInfo = pd.read_csv("studentInfo.csv")
studentRegistration = pd.read_csv("studentRegistration.csv")
studentVle = pd.read_csv("studentVle.csv")
vle = pd.read_csv("vle.csv")

In [436]:
studentInfo.head()
# the main table because it contains the final results

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [437]:
studentInfo.count()['code_module']

32593

In [438]:
studentInfo.groupby("final_result").count()['code_module']

final_result
Distinction     3024
Fail            7052
Pass           12361
Withdrawn      10156
Name: code_module, dtype: int64

## EDA -- break into dimensions

In [439]:
gender_group = studentInfo.groupby(["gender",'final_result']).count()['code_module']
gender_group / gender_group.groupby(level=0).sum()
# no big difference

gender  final_result
F       Distinction     0.094714
        Fail            0.210830
        Pass            0.389659
        Withdrawn       0.304797
M       Distinction     0.091189
        Fail            0.220923
        Pass            0.370685
        Withdrawn       0.317203
Name: code_module, dtype: float64

In [440]:
region_group = studentInfo.groupby(["region",'final_result']).count()['code_module']
region_group / region_group.groupby(level=0).sum()
# London Region and North Western Region has a high Fail or Withdrawn rate

region                final_result
East Anglian Region   Distinction     0.097904
                      Fail            0.208982
                      Pass            0.391916
                      Withdrawn       0.301198
East Midlands Region  Distinction     0.084567
                                        ...   
West Midlands Region  Withdrawn       0.353602
Yorkshire Region      Distinction     0.082253
                      Fail            0.223829
                      Pass            0.366401
                      Withdrawn       0.327517
Name: code_module, Length: 52, dtype: float64

In [441]:
highest_education_group = studentInfo.groupby(["highest_education",'final_result']).count()['code_module']
highest_education_group / highest_education_group.groupby(level=0).sum()
# Lower Than A Level and No Formal quals have a higher Fail or Withdrawal rate

highest_education            final_result
A Level or Equivalent        Distinction     0.106515
                             Fail            0.192738
                             Pass            0.413813
                             Withdrawn       0.286935
HE Qualification             Distinction     0.147357
                             Fail            0.167019
                             Pass            0.414376
                             Withdrawn       0.271247
Lower Than A Level           Distinction     0.055252
                             Fail            0.260374
                             Pass            0.333257
                             Withdrawn       0.351117
No Formal quals              Distinction     0.046110
                             Fail            0.273775
                             Pass            0.250720
                             Withdrawn       0.429395
Post Graduate Qualification  Distinction     0.281150
                             Fail       

In [442]:
imd_band_group = studentInfo.groupby(["imd_band",'final_result']).count()['code_module']
imd_band_group / imd_band_group.groupby(level=0).sum()
# It's obvious that when the students live in a richer place, it's easier for them to finish and pass.

imd_band  final_result
0-10%     Distinction     0.050740
          Fail            0.276654
          Pass            0.300815
          Withdrawn       0.371791
10-20     Distinction     0.054323
                            ...   
90-100%   Withdrawn       0.258675
?         Distinction     0.179118
          Fail            0.130513
          Pass            0.477948
          Withdrawn       0.212421
Name: code_module, Length: 44, dtype: float64

In [443]:
age_band_group = studentInfo.groupby(["age_band",'final_result']).count()['code_module']
age_band_group / age_band_group.groupby(level=0).sum()
# The younger the group is, the higher failure or withdrwan rate they have

age_band  final_result
0-35      Distinction     0.081198
          Fail            0.227990
          Pass            0.369116
          Withdrawn       0.321696
35-55     Distinction     0.118732
          Fail            0.189971
          Pass            0.402841
          Withdrawn       0.288455
55<=      Distinction     0.189815
          Fail            0.134259
          Pass            0.425926
          Withdrawn       0.250000
Name: code_module, dtype: float64

## Data Cleaning & Feature Engineering

In [444]:
assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19,10.0
1,AAA,2013J,1753,TMA,54,20.0
2,AAA,2013J,1754,TMA,117,20.0
3,AAA,2013J,1755,TMA,166,20.0
4,AAA,2013J,1756,TMA,215,30.0


In [445]:
studentAssessment.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1759,6516,51,0,48
1,1758,6516,17,0,60
2,1761,6516,164,0,61
3,1760,6516,116,0,63
4,1762,6516,210,0,77


#### For these two tables, I want to know for each student and course, what's the average date in advance when they submit the assignment. I don't include "weight" or "score" because they are part of the final result.

In [446]:
assessments["date"].value_counts()
# there are few '?' in the table, so I just dropped the rows

222    15
229    14
?      11
241     9
236     8
       ..
195     1
88      1
165     1
136     1
146     1
Name: date, Length: 75, dtype: int64

In [447]:
assessments = assessments[assessments["date"] != '?']

In [448]:
assessment_submission = pd.merge(left=assessments, right=studentAssessment, on=['id_assessment'], how="left")

In [449]:
assessment_submission["date"] = pd.to_numeric(assessment_submission["date"], downcast="float")

In [450]:
# to measure how many days in advance when students submitted the assessment
assessment_submission['advanced_date_submitted'] = assessment_submission['date_submitted'] - assessment_submission['date']

In [451]:
assessment_submission["advanced_date_submitted"].value_counts()
# some assignments were submitted really late 

 0.0      29469
-1.0      25846
 2.0      19318
-2.0      10455
 3.0       7827
          ...  
 334.0        1
 88.0         1
 132.0        1
 198.0        1
 115.0        1
Name: advanced_date_submitted, Length: 390, dtype: int64

In [452]:
assessment_submission_average = assessment_submission.groupby(by=["code_module", "code_presentation", "id_student"]).mean()['advanced_date_submitted']

In [453]:
assessment_submission_average.head()

code_module  code_presentation  id_student
AAA          2013J              11391.0       -1.8
                                28400.0        0.0
                                31604.0       -2.0
                                32885.0       11.4
                                38053.0        2.0
Name: advanced_date_submitted, dtype: float64

In [454]:
merge_1 = pd.merge(left=studentInfo, right=assessment_submission_average, on=["code_module","code_presentation", "id_student"], how="left")

In [455]:
sum(merge_1['advanced_date_submitted'].isna())

6751

In [456]:
merge_1

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,advanced_date_submitted
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-1.800000
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,0.000000
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,NaN
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-2.000000
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,11.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32588,GGG,2014J,2640965,F,Wales,Lower Than A Level,10-20,0-35,0,30,N,Fail,NaN
32589,GGG,2014J,2645731,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,Distinction,-36.222222
32590,GGG,2014J,2648187,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,Pass,-45.333333
32591,GGG,2014J,2679821,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N,Withdrawn,-79.500000


In [457]:
courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [458]:
studentRegistration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159,?
1,AAA,2013J,28400,-53,?
2,AAA,2013J,30268,-92,12
3,AAA,2013J,31604,-52,?
4,AAA,2013J,32885,-176,?


#### For these two tables, "module_presentation_length" & "date_registration" might be useful in the model

In [459]:
merge_2 = pd.merge(left=merge_1, right=courses, on=["code_module","code_presentation"], how="left")

In [460]:
merge_3 = pd.merge(left=merge_2, right=studentRegistration, on=["code_module","code_presentation", "id_student"], how="left")

In [461]:
merge_3 = merge_3.drop(columns="date_unregistration")

In [462]:
merge_3.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,advanced_date_submitted,module_presentation_length,date_registration
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-1.8,268,-159
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,0.0,268,-53
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,NaN,268,-92
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-2.0,268,-52
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,11.4,268,-176


In [463]:
studentVle.head()

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


In [464]:
studentVle["sum_click"].value_counts()

1       5113910
2       1929023
3        965277
4        612858
5        415139
         ...   
403           1
402           1
401           1
395           1
6977          1
Name: sum_click, Length: 498, dtype: int64

In [465]:
vle.head()

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,?,?
1,546712,AAA,2013J,oucontent,?,?
2,546998,AAA,2013J,resource,?,?
3,546888,AAA,2013J,url,?,?
4,547035,AAA,2013J,resource,?,?


In [466]:
vle["week_from"].value_counts()

?     5243
18      91
1       84
28      63
9       52
      ... 
25      24
6       17
7       15
17       4
0        2
Name: week_from, Length: 31, dtype: int64

In [467]:
vle["week_to"].value_counts()

?     5243
18      91
1       80
28      63
9       54
      ... 
25      24
7       17
6       17
17       4
0        2
Name: week_to, Length: 31, dtype: int64

#### For these two tables, "activity_type" is useful but it is hard to take aggregation, and "week_from" & "week_to" have too many "?" values, so I just include the sum of "sum_click"

In [468]:
vle_click = studentVle.groupby(by=["code_module", "code_presentation", "id_student"]).sum()["sum_click"]

In [469]:
vle_click

code_module  code_presentation  id_student
AAA          2013J              11391          934
                                28400         1435
                                30268          281
                                31604         2158
                                32885         1034
                                              ... 
GGG          2014J              2640965         41
                                2645731        893
                                2648187        312
                                2679821        275
                                2684003        616
Name: sum_click, Length: 29228, dtype: int64

In [470]:
merge_4 = pd.merge(left=merge_3, right=vle_click, on=["code_module", "code_presentation", "id_student"], how="left")

In [471]:
df_model = merge_4.drop(columns=["code_module", "code_presentation", "id_student"])
df_model.head()

,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,advanced_date_submitted,module_presentation_length,date_registration,sum_click
0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-1.8,268,-159,934.0
1,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,0.0,268,-53,1435.0
2,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,NaN,268,-92,281.0
3,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-2.0,268,-52,2158.0
4,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,11.4,268,-176,1034.0


#### After combining all the variables in one table, I will start transforming them

In [472]:
# change all teh "?" to NaN for future convenience
df_model = df_model.replace("?", float("NaN"))

In [473]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32593 entries, 0 to 32592
Data columns (total 13 columns):
gender                        32593 non-null object
region                        32593 non-null object
highest_education             32593 non-null object
imd_band                      31482 non-null object
age_band                      32593 non-null object
num_of_prev_attempts          32593 non-null int64
studied_credits               32593 non-null int64
disability                    32593 non-null object
final_result                  32593 non-null object
advanced_date_submitted       25842 non-null float64
module_presentation_length    32593 non-null int64
date_registration             32548 non-null object
sum_click                     29228 non-null float64
dtypes: float64(2), int64(3), object(8)
memory usage: 3.5+ MB


In [474]:
# for sum_clicks, when there is missing value, it means it's 0
df_model['sum_click'].fillna(0, inplace=True)

# for advanced_date_submitted and date_registration, I impute the missing value with median
df_model['advanced_date_submitted'] = df_model['advanced_date_submitted'].fillna(df_model['advanced_date_submitted'].median())
df_model['date_registration'] = df_model['date_registration'].fillna(df_model['date_registration'].median())

# for categorical variables, I impute the missing value with the most frequent levels
df_model[['gender','region','highest_education','imd_band','age_band','disability']] \
= df_model[['gender','region','highest_education','imd_band','age_band','disability']]\
.apply(lambda x:x.fillna(x.value_counts().index[0]))


In [475]:
# create dummies for categorical variables
df_model = pd.get_dummies(df_model, columns = ['gender','region','highest_education','imd_band','age_band','disability'], drop_first=True)

In [476]:
df_model['final_result_dummy'] = np.where(((df_model['final_result']=='Pass')|(df_model['final_result']=='Distinction')),1,0)

In [477]:
df_model.drop(columns='final_result', inplace=True)

In [478]:
df_model.head(5)

,num_of_prev_attempts,studied_credits,advanced_date_submitted,module_presentation_length,date_registration,sum_click,gender_M,region_East Midlands Region,region_Ireland,region_London Region,region_North Region,region_North Western Region,region_Scotland,region_South East Region,region_South Region,region_South West Region,region_Wales,region_West Midlands Region,region_Yorkshire Region,highest_education_HE Qualification,highest_education_Lower Than A Level,highest_education_No Formal quals,highest_education_Post Graduate Qualification,imd_band_10-20,imd_band_20-30%,imd_band_30-40%,imd_band_40-50%,imd_band_50-60%,imd_band_60-70%,imd_band_70-80%,imd_band_80-90%,imd_band_90-100%,age_band_35-55,age_band_55<=,disability_Y,final_result_dummy
0,0,240,-1.800000,268,-159,934.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1
1,0,60,0.000000,268,-53,1435.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
2,0,60,-0.583333,268,-92,281.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0
3,0,60,-2.000000,268,-52,2158.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1
4,0,60,11.400000,268,-176,1034.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


## Models

In [479]:
X = df_model.drop(columns='final_result_dummy')
y = df_model['final_result_dummy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [480]:
def model_metrics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    print("Precision:", metrics.precision_score(y_test, y_pred))
    print("Recall:", metrics.recall_score(y_test, y_pred))
    print("R2 score:", metrics.r2_score(y_test, y_pred))
    print("F1 score:", metrics.f1_score(y_test, y_pred))
    print("ROC AUC score:", metrics.roc_auc_score(y_test, y_pred))

### Logistic Regression

In [486]:
model = sm.Logit(y_train, X_train.astype(float), max_iter=1000, penalty='l1')
result = model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.470544
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:     final_result_dummy   No. Observations:                22815
Model:                          Logit   Df Residuals:                    22780
Method:                           MLE   Df Model:                           34
Date:                Thu, 24 Dec 2020   Pseudo R-squ.:                  0.3196
Time:                        12:00:59   Log-Likelihood:                -10735.
converged:                       True   LL-Null:                       -15778.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
num_of_prev_attempts                  

In [482]:
model_metrics(logreg, X_test, y_test)

Accuracy: 0.7879934546942114
Precision: 0.8146498391487256
Recall: 0.7131715771230502
R2 score: 0.14932134710593403
F1 score: 0.7605406029802472
ROC AUC score: 0.7840363891039506


### Random Forest

In [484]:
randomfst = RandomForestClassifier(random_state=10)
randomfst.fit(X_train,y_train)
pd.Series(randomfst.feature_importances_,index=X.columns).sort_values(ascending=False).head(15)

sum_click                               0.395006
advanced_date_submitted                 0.217238
date_registration                       0.083198
module_presentation_length              0.045892
studied_credits                         0.037221
gender_M                                0.017474
highest_education_Lower Than A Level    0.015366
num_of_prev_attempts                    0.013779
age_band_35-55                          0.012671
imd_band_20-30%                         0.008507
imd_band_30-40%                         0.007841
region_Scotland                         0.007837
disability_Y                            0.007719
highest_education_HE Qualification      0.007680
imd_band_40-50%                         0.007608
dtype: float64

In [485]:
model_metrics(randomfst, X_test, y_test)
# this result is better

Accuracy: 0.8272652894252404
Precision: 0.8008221993833504
Recall: 0.8440207972270364
R2 score: 0.30690002665794625
F1 score: 0.8218542347853601
ROC AUC score: 0.8281514292218096
